In [1]:
from sparkdl import readImages
from pyspark.sql.functions import lit

Using TensorFlow backend.


In [2]:
img_dir = "/home/ec2-user/Data/kaggle-planet-amazon/"

train_df = readImages(img_dir + "/train-jpg")
test_df = readImages(img_dir + "/test-jpg")

In [3]:
train_df.printSchema()

root
 |-- filePath: string (nullable = false)
 |-- image: struct (nullable = true)
 |    |-- mode: string (nullable = false)
 |    |-- height: integer (nullable = false)
 |    |-- width: integer (nullable = false)
 |    |-- nChannels: integer (nullable = false)
 |    |-- data: binary (nullable = false)



In [4]:
train_df_train, train_df_valid = train_df.randomSplit([0.6, 0.4])

In [6]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from sparkdl import DeepImageFeaturizer

featurizer = DeepImageFeaturizer(inputCol="image", outputCol="features", modelName="InceptionV3")
lr = LogisticRegression(maxIter=20, regParam=0.05, elasticNetParam=0.3, labelCol="label")
p = Pipeline(stages=[featurizer, lr])
p_model = p.fit(train_df_train)

A local file was found, but it seems to be incomplete or outdated because the md5 file hash does not match the original value of bcbd6486424b2319ff4ef7d526e38f63 so we will re-download the data.
86016000/87910968 [============================>.] - ETA: 0sINFO:tensorflow:Froze 376 variables.
Converted 376 variables to const ops.


Py4JJavaError: An error occurred while calling o136.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 5, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark220/python/lib/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/opt/spark220/python/lib/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/spark220/python/lib/pyspark.zip/pyspark/serializers.py", line 220, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/opt/spark220/python/lib/pyspark.zip/pyspark/serializers.py", line 138, in dump_stream
    for obj in iterator:
  File "/opt/spark220/python/lib/pyspark.zip/pyspark/serializers.py", line 209, in _batched
    for item in iterator:
  File "<string

In [ ]:
predictions = p_model.transform(train_df_valid)

predictions.select("filePath", "prediction").show(truncate=False)

img_dir = "/home/ec2-user/Pictures/personalities/"

#Read images and Create training & test DataFrames for transfer learning
jobs_df = readImages(img_dir + "/jobs").withColumn("label", lit(1))
zuckerberg_df = readImages(img_dir + "/zuckerberg").withColumn("label", lit(0))
jobs_train, jobs_test = jobs_df.randomSplit([0.6, 0.4])
zuckerberg_train, zuckerberg_test = zuckerberg_df.randomSplit([0.6, 0.4])

#dataframe for training a classification model
train_df = jobs_train.unionAll(zuckerberg_train)

#dataframe for testing a classification model
test_df = jobs_test.unionAll(zuckerberg_test)

from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from sparkdl import DeepImageFeaturizer

featurizer = DeepImageFeaturizer(inputCol="image", outputCol="features", modelName="InceptionV3")
lr = LogisticRegression(maxIter=20, regParam=0.05, elasticNetParam=0.3, labelCol="label")
p = Pipeline(stages=[featurizer, lr])
p_model = p.fit(train_df)

predictions = p_model.transform(test_df)

predictions.select("filePath", "prediction").show(truncate=False)